In [1]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.api import ai_client
from src.utils import *
from src.const import *
from src.plots import *
from src.signal_providers.signal_manager import *
from src.news_providers.news_api_provider import *
from src.agents.news_filtering_agent import *
from src.agents.trading_agent import *
from src.signal_providers.bybit_bid_ask import BybitBidAskProvider

%load_ext autoreload
%autoreload 2

sns.set_theme(style="darkgrid")

In [2]:
coin = "ETH"
bid_ask_provider = BybitBidAskProvider()

day_history = []
heights = []

def reload_data():
    global day_history, heights

    day_history, bid_ask = map_async(lambda: get_12h_history(coin), lambda: bid_ask_provider.get_data(coin))
    
    heights = [[x["price"], x["amount"]] for x in bid_ask["bid"]]
    ask_heights = [[x["price"], -x["amount"]] for x in bid_ask["ask"]]
    heights.extend(ask_heights)

In [5]:
reload_data()
    
def plot_data(a):
    plt.hist([x[1] for x in heights if abs(x[1]) < a], density=True, bins=100)

    plt.show()

button = widgets.Button(
    description='Reload',
    button_style='info',
    tooltip='Click to reset the plot',
    icon='refresh'
)

slider = widgets.FloatSlider(
    value=101.0,
    min=0.01,
    max=3000.0,
    step=0.1,
    description='Amplitude:',
    continuous_update=False
)

def on_button_clicked(b):
    reload_data()
    orig = slider.value
    slider.value = orig + 1e-6
    slider.value = orig

button.on_click(on_button_clicked)

ui = widgets.VBox([slider, button])
display(ui)

# Bind the slider to the plot function using interactive
out = widgets.interactive_output(plot_data, {'a': slider})
display(out)

Output()

In [7]:
reload_data()
    
def plot_data(a):
    plot_candles_overlay(day_history, heights=heights, display_range=(-a, a), figsize=(20, 8))

button = widgets.Button(
    description='Reload',
    button_style='info',
    tooltip='Click to reset the plot',
    icon='refresh'
)

slider = widgets.FloatSlider(
    value=100.0,
    min=0.01,
    max=3000.0,
    step=0.1,
    description='Amplitude:',
    continuous_update=False
)

def on_button_clicked(b):
    reload_data()
    orig = slider.value
    slider.value = orig + 1e-6
    slider.value = orig

button.on_click(on_button_clicked)

ui = widgets.VBox([slider, button])
display(ui)

# Bind the slider to the plot function using interactive
out = widgets.interactive_output(plot_data, {'a': slider})
display(out)

Output()